In [77]:
import os
import torch
from tqdm import tqdm

In [78]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("your_model_name")
model = AutoModelForCausalLM.from_pretrained("your_model_name", device_map='auto',torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.80s/it]


In [79]:
import pandas as pd
df = pd.read_csv('your_test_data.csv')
str(df.head())

"   id          rgno                                      context  pattern  \\\n0   0  105510003412  ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ    11015   \n1   1  105510003412  ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ    11015   \n2   2  105510003412  ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ    11015   \n3   3  105510003412  ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ    11015   \n4   4  105510003412  ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ    11015   \n\n               question                           legal_act condition  \n0    ['สหชา อินทร์ไชย']                          การขอวีซ่า       NaN  \n1      ['สุเมธ คุ้มคง']                        การค้ำประกัน       NaN  \n2     ['ภควัฒน์ ผาสุข']  ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ       NaN  \n3  ['อชิระ อาจนาเสียว']                              ธนาคาร       NaN  \n4   ['ณัฐวีร์ แนวกำพล']                          มัคคุเทศน์       NaN  "

In [97]:
def formatting_prompts_func(example):
    output_texts = []

    for i in range(len(example["context"])):
        
        try:
            total = len(eval(question))
        except:
            total = "don't know"
        
        messages = [
    {
        "role": "system",
        "content": "You are a Thai linguist who knows about the Thai language. Your job is to verify the authority of the Fraud Detection Committee based on the signatures provided in context. The committee whose signature is in the committee list.\nFormat task that provides:\n1. Context\nDescription of context: This includes the name of the committee and conditions.\nCommittee is a group of members with the authority, granted fairly, to sign documents.\nThe condition often takes the form of specifying the required number of committee members to sign, determining which committee group to draw from, and requiring someone to sign if the amount exceeds X baht \n2. Signs_committee\nDescription of signs_committee: The group of people who sign this document.\n3. number_of_committee \nDescription of number_of_committee: The number of signs_committee \n Instruction: Helps check whether the committee's signatures match the conditions stated earlier in the context. You should respond with 0 if they don't match, and 1 if they do. don't respond in string"
    },
    {
        "role": "user",
        "content": f"context : {example['context'][i]}.\nSigns_committee : {example['question'][i]}\n number_of_committee:{total}"
    },
    {"role": "assistant", "content": str(example["answer"][i])},
]
        output_texts.append(messages)
    return output_texts

In [86]:
# Apply the formatting_prompts_func function to your data
formatted_data = formatting_prompts_func(df)

In [87]:
outputs = []
for example in tqdm(formatted_data):
    gen_input = tokenizer.apply_chat_template(example, add_generation_prompt=True, return_tensors="pt").to(model.device)
    out = model.generate(gen_input, max_new_tokens=1)
    output = tokenizer.decode(out[0]).split('|im_start|>assistant')[-1].split('<|im_end|>')[0].strip().split('<eos>')[0]
    outputs.append(output)

100%|██████████| 5835/5835 [07:15<00:00, 13.39it/s]


In [88]:
submission = pd.DataFrame({"id":list(range(len(outputs))),"answer":outputs})

In [89]:
submission.to_csv("submission.csv",index=False)